# HW2: Морфологические парсеры
### Такташева Катя
#### Вспомогательные функции

In [1]:
import pandas as pd
import numpy as np

In [2]:
COLUMNS = ['word', 'POS']


def to_pandas(text):
    """
    converts annotated text to pandas dataframe
    """
    words = text.split()
    df = [word.split('_') for word in words]
    return pd.DataFrame(df, columns=COLUMNS)

В качестве списка тэгов возьмем самый узкий: не будет разделять все формы глаголов (инфинитив, причастия, деепричастия), формы прилагательных (некоторые таггеры размечают сравнительные и краткие формы), местоимения и др.

In [3]:
def convert_names(tag, lang='ru'):
    """
    convert all tags to a uniform standart
    """
    transform = {}
    tag = str(tag)
    if tag not in transform:
        if 'CONJ' in tag or 'CC' in tag:
            transform[tag] = 'CONJ'
        elif tag == 'PROPN':
            transform[tag] = 'NOUN'
        elif tag.startswith('ADJ') or tag in ['A', 'JJ', 'COMP']:
            transform[tag] = 'ADJ'
        elif 'PRO' in tag or tag in ['PRP', 'EX', 'PRP$', 'PRP', 'WP']:
            transform[tag] = 'PRON'
        elif tag.startswith('V') or tag == 'MD':
            transform[tag] = 'VERB'
        elif tag in ['AUX', 'INFN', 'PRED', 'GRND']:
            transform[tag] = 'VERB'
        elif tag in ['ADP', 'PR', 'IN', 'RP']:
            transform[tag] = 'PREP'
        elif tag.startswith('AD') or tag.endswith('RB'):
            transform[tag] = 'ADVB'
        elif tag.startswith('NN') or tag == 'S':
            transform[tag] = 'NOUN'
        elif tag in ['PART', 'PRT', 'TO']:
            transform[tag] = 'PRCL'
        elif tag == 'DT':
            transform[tag] = 'DET'
    if lang == 'ru' and 'DET' not in transform:
        transform['DET'] = 'PRON'
    return transform[tag] if tag in transform else tag

### Load texts

In [4]:
with open('ru.txt', encoding='utf-8') as fid:
    ru_text = fid.read()

In [5]:
with open('en.txt', encoding='utf-8') as fid:
    en_text = fid.read()

In [6]:
print(f'Russian: {len(ru_text.split())} words')
print(f'English: {len(en_text.split())} words')

Russian: 168 words
English: 131 words


# Russian

In [7]:
ru_text

'Ты белых лебедей кормила , Откинув тяжесть черных кос … Я рядом плыл ; сошлись кормила ; Закатный луч был странно кос . Вдруг лебедей метнулась пара … Не знаю , чья была вина … Закат замлел за дымкой пара , Алея , как поток вина . Мгновенья двигались и стали , Лишь ты паришь , свой свет струя … Меж тем в реке – из сизой стали Влачится за струей струя . Небесное светило было закрыто дождевыми тучами , а еще вчера оно так ярко и ласково нам светило . Если б мыло приходило по утрам ко мне в кровать и само меня бы мыло , хорошо бы это было . Мы ели , ели ершей у ели . Их еле-еле у ели доели . Вам нужно тщательнее следить за вашим рабочим местом . Рабочим было трудно завершить строительство во время пандемии из-за закрытия всех магазинов . Плывут по озеру пироги – На остров нет другой дороги . Любишь кушать пироги – Печь их маме помоги . '

**Что тут сложного?**
Как тут, так и в английском тексте присутсвует много морфологически омонимичных слов - т.е. слов разных частей речи, которые сложно распарсить на, собственно, части речи без контекста. Поскольку большинство тэггеров не смотрят на контекст, задача тэггинга таких единиц предстваляет проблему.

**Примеры слов:**

В русском:
- *струя* (дееприч., ср. свет струя) - *струя* (сущ., ср. струя реки)
- *ели* (гл. мы ели суп) - *ели* (сущ. им/вин.п. мн.ч., ср. зеленые ели)
- *рабочим* (сущ. д.п.мн.ч./тв.п.ед.ч. , ср. гордиться рабочим, дать рабочим задание) - *рабочим* (прил. тв.п.ед.ч. он гордился своим рабочим местом)
- тут еще много других

В английском:
- *cook* (гл., ср. to cook fish) - *cook* (прил., ср. a cook book) - *cook* (сущ. ср. he's a great cook)
- *well* (сущ., to fall in a well) - *well* (наречие, ср. well done)
- и другие

#### Hand-parsed

In [8]:
with open('ru_text.txt', encoding='utf-8') as fid:
    ru_answers = fid.read()

In [9]:
ru_df = to_pandas(ru_answers)
ru_df

,word,POS
0,Ты,PRON
1,белых,ADJ
2,лебедей,NOUN
3,кормила,VERB
4,",",PUNCT
...,...,...
163,Печь,VERB
164,их,PRON
165,маме,NOUN
166,помоги,VERB


## Pymorphy

In [10]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [11]:
pymorphy_df = []
for token in [morph.parse(word)[0] for word in ru_text.split()]:
    word = token.word
    POS = token.tag.POS
    if POS is None:
        POS = 'PUNCT'
    pymorphy_df.append((word, POS))

In [12]:
pymorphy_df = pd.DataFrame(pymorphy_df, columns=COLUMNS)
pymorphy_df

,word,POS
0,ты,NPRO
1,белых,ADJF
2,лебедей,NOUN
3,кормила,VERB
4,",",PUNCT
...,...,...
163,печь,NOUN
164,их,NPRO
165,маме,NOUN
166,помоги,VERB


## MyStem

In [13]:
from pymystem3 import Mystem
m = Mystem()

In [14]:
mystem_df = []
for token in [m.analyze(word.strip()) for word in ru_text.split()]:
    if 'analysis' in token[0]:
        word = token[0]['text']
        POS = token[0]['analysis'][0]['gr'].replace('=', ',').split(',', maxsplit=1)[0]
    elif token[0]['text'] != ' ':
        word = token[0]['text']
        POS = 'PUNCT'
    mystem_df.append((word, POS))

In [15]:
mystem_df = pd.DataFrame(mystem_df, columns=COLUMNS)
mystem_df

,word,POS
0,Ты,SPRO
1,белых,A
2,лебедей,S
3,кормила,V
4,",\n",PUNCT
...,...,...
163,Печь,S
164,их,SPRO
165,маме,S
166,помоги,V


## Natasha

In [16]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

In [17]:
segmenter = Segmenter()
text = Doc(ru_text)
text.segment(segmenter)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
text.tag_morph(morph_tagger)

In [18]:
natasha_df = []
for sent in text.sents:
    parsed = sent.morph.tokens
    for token in parsed:
        word = token.text
        POS = token.pos
        natasha_df.append((word, POS))

In [19]:
natasha_df = pd.DataFrame(natasha_df, columns=COLUMNS)
natasha_df

,word,POS
0,Ты,PRON
1,белых,ADJ
2,лебедей,NOUN
3,кормила,VERB
4,",",PUNCT
...,...,...
163,Печь,PROPN
164,их,PRON
165,маме,NOUN
166,помоги,VERB


## Comparison

Приводим к одному виду:

In [20]:
pymorphy_df['POS'] = pymorphy_df['POS'].apply(convert_names)
mystem_df['POS'] = mystem_df['POS'].apply(convert_names)
natasha_df['POS'] = natasha_df['POS'].apply(convert_names)

In [21]:
from sklearn.metrics import accuracy_score


print(f"PyMorphy: {accuracy_score(pymorphy_df['POS'].tolist(), ru_df['POS'].tolist())}")
print(f"MyStem: {accuracy_score(mystem_df['POS'].tolist(), ru_df['POS'].tolist())}")
print(f"Natasha: {accuracy_score(natasha_df['POS'].tolist(), ru_df['POS'].tolist())}")

PyMorphy: 0.9345238095238095
MyStem: 0.8333333333333334
Natasha: 0.8214285714285714


# English

In [22]:
en_text

"The other day I have bought a new cook book. It is awesome. There are lots of great recipes in it and I can't wait to cook them all. \nBefore you travel in a different country you need to make sure you book a hotel, else you might end up on the street.\nMy neighbours told us a scary story about a woman who fell in the well. \nI was not feeling well after hearing it.\nYou need to bow every time the queen walks in or it will cost you your head.\nHe always wanted to learn how to shoot an arrow, but didn't manage to buy a bow.\nHe was our only hope in getting out of town.\nLet's hope we can save some money for a ride home."

#### Hand-parsed

In [23]:
with open('en_text.txt', encoding='utf-8') as fid:
    en_answers = fid.read()

In [24]:
en_df = to_pandas(en_answers)
en_df

,word,POS
0,The,DET
1,other,ADJ
2,day,NOUN
3,I,PRON
4,have,VERB
...,...,...
141,for,PREP
142,a,DET
143,ride,NOUN
144,home,NOUN


## SpaCy

In [25]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()

/Users/katya/opt/anaconda3/lib/python3.7/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [26]:
spacy_df = []
for token in nlp(en_text):
    word = token.text
    POS = token.pos_
    if POS != 'SPACE':
        spacy_df.append((word, POS))

In [27]:
spacy_df = pd.DataFrame(spacy_df, columns=COLUMNS)
spacy_df

,word,POS
0,The,DET
1,other,ADJ
2,day,NOUN
3,I,PRON
4,have,AUX
...,...,...
141,for,ADP
142,a,DET
143,ride,NOUN
144,home,NOUN


## NLTK

In [28]:
import nltk
from string import punctuation

In [29]:
nltk_df = []
for i in nltk.word_tokenize(en_text):
    parsed = nltk.pos_tag([i],tagset='universal')
    word = parsed[0][0]
    POS = parsed[0][1]
    if POS in punctuation:
        POS = 'PUNCT'
    nltk_df.append((word, POS))

In [30]:
nltk_df = pd.DataFrame(nltk_df, columns=COLUMNS)
nltk_df

,word,POS
0,The,DET
1,other,ADJ
2,day,NOUN
3,I,PRON
4,have,VERB
...,...,...
141,for,ADP
142,a,DET
143,ride,NOUN
144,home,NOUN


## Flair

In [31]:
from flair.data import Sentence
from flair.models import SequenceTagger
import re

tagger = SequenceTagger.load('pos')

2020-10-16 18:04:49,993 loading file /Users/katya/.flair/models/en-pos-ontonotes-v0.5.pt


In [32]:
flair_df = []
for sent in nltk.sent_tokenize(en_text):
    sentence = Sentence(sent)
    tagger.predict(sentence)
    parsed = sentence.to_tagged_string().split()
    for i in range(0, len(parsed)-1, 2):
        word, POS = parsed[i], re.sub('<|>', '', parsed[i+1])
        if POS in punctuation:
            POS = 'PUNCT'
        flair_df.append((word, POS))

In [33]:
flair_df = pd.DataFrame(flair_df, columns=COLUMNS)
flair_df

,word,POS
0,The,DT
1,other,JJ
2,day,NN
3,I,PRP
4,have,VBP
...,...,...
141,for,IN
142,a,DT
143,ride,NN
144,home,RB


## Comparison
Приводим к одному виду:

In [34]:
spacy_df['POS'] = spacy_df['POS'].apply(convert_names, lang='en')
nltk_df['POS'] = nltk_df['POS'].apply(convert_names, lang='en')
flair_df['POS'] = flair_df['POS'].apply(convert_names, lang='en')

In [35]:
print(f"SpaCy: {accuracy_score(spacy_df['POS'].tolist(), en_df['POS'].tolist())}")
print(f"NLTK: {accuracy_score(nltk_df['POS'].tolist(), en_df['POS'].tolist())}")
print(f"Flair: {accuracy_score(flair_df['POS'].tolist(), en_df['POS'].tolist())}")

SpaCy: 0.9931506849315068
NLTK: 0.815068493150685
Flair: 0.9726027397260274


## Classifier improvement (?)

Load data:

In [36]:
def get_smooth_bins(data, n=1000):
    smooth = []
    for label in ['pos', 'neg']:
        smooth.append(data[data['class']==label].sample(n//2))
    return pd.concat(smooth, ignore_index=True)

In [37]:
reviews = pd.read_csv('reviews.csv', delimiter=',')
reviews = get_smooth_bins(reviews, 10000)
reviews

,film_id,text,class
0,978935,"фильм « исчезновение » я ждать , так как трейл...",pos
1,1207666,"давно не писать рецензия на фильм . 4 год , ес...",pos
2,808639,"' дуpaк ' из тот фильмoть , кoтopый бить oбухo...",pos
3,301,исполниться 20 год « матрица » — коктейль от р...,pos
4,680149,основной маркёр это фильм – он оригинальный на...,pos
...,...,...,...
9995,797840,не радовать . \n\n зачем главное герой сделать...,neg
9996,273302,фильм от режиссёр великий произведение кинемат...,neg
9997,5492,« можно стереть любовь из память . выкинуть из...,neg
9998,381,фильм - взгяд на происходить в жизнь событие ч...,neg


In [38]:
reviews['class'].value_counts()

neg    5000
pos    5000
Name: class, dtype: int64

In [39]:
train, test = np.split(reviews.sample(frac=1), [int(.80*len(reviews))])

In [40]:
print(f'Train sentences: {len(train)}')
print(f'Test sentences: {len(test)}')

Train sentences: 8000
Test sentences: 2000


In [41]:
train['class'].value_counts()

pos    4001
neg    3999
Name: class, dtype: int64

In [42]:
test['class'].value_counts()

neg    1001
pos     999
Name: class, dtype: int64

Попробуем улучшить качество классификатора из прошлой домашки, будем брать биграммы и триграммы следующих паттернов:
- (не +) прилагательное ((не) плохой, (не) хороший)
- (не +) наречие ((не) плохо, (не) хорошо)
- (не +) наречие + прилагательное ((не) очень плохой)
- (не +) наречие + наречие ((не) очень хорошо)

Как кажется, это самые оценточные коллокации, которые должны самый большой вклад в "точность" словаря

In [106]:
def find_grams(text):
    doc = Doc(text)
    doc.segment(segmenter)
    n_grams = []
    for i, sent in enumerate(doc.sents):
        tokens = sent.text.split()
        analysis = [morph.parse(t)[0].tag.POS for t in tokens]
        for j, an in enumerate(analysis[:-1]):
            if tokens[j] == 'не':
                if analysis[j+1] == 'ADJF':
                    n_grams.append((' '.join((tokens[j], tokens[j+1]))))
                elif analysis[j+1] == 'ADVB':
                    try:
                        if analysis[j+2] == 'ADJF'or analysis[j+2] == 'ADVB':
                            n_grams.append((' '.join((tokens[j], tokens[j+1], tokens[j+2]))))
                    except IndexError:
                        pass
            elif an == 'ADJF':
                if analysis[j+1] == 'NOUN':
                    n_grams.append(' '.join((tokens[j], tokens[j+1])))
            elif an == 'ADVB':
                if analysis[j+1] == 'ADJF' or analysis[j+1] == 'ADVB':
                    n_grams.append(' '.join((tokens[j], tokens[j+1])))
    return n_grams

In [107]:
from tqdm.auto import tqdm
from collections import Counter
from nltk.corpus import stopwords
ru_stopwords = set(stopwords.words('russian'))

In [108]:
def preprocess(text, lemmatize=False):
    words = []
    for word in text.split():
        if word.isalpha() and word not in ru_stopwords:
            if lemmatize:
                word = morph.parse(word)[0].normal_form
            words.append(word)
    return words

In [109]:
def create_dict(pos_words, neg_words):
    tone_dict = {}
    for word in pos_words:
        tone_dict[word] = 'pos'
    for word in neg_words:
        tone_dict[word] = 'neg'
        
    return tone_dict


def get_tone_words(data, min_count=10):
    
    dic = {'pos': Counter(),
           'neg': Counter()}
    
    print('Computing frequency dict...')
    for idx, review in tqdm(data.iterrows(), total=len(data)):
        dic[review['class']] += Counter(preprocess(review['text']))
        dic[review['class']] += Counter(find_grams(review['text']))
            
    negative = set([i[0] for i in dic['neg'].most_common() if i[1] >= min_count])
    positive = set([i[0] for i in dic['pos'].most_common() if i[1] >= min_count])
    
    
    
    intersect = positive.intersection(negative)
    for i in intersect:
        positive.discard(i)
        negative.discard(i)
    
    #min_size = min((len(positive), len(negative)))  # по-хорошему надо сравнять размеры классов, но
    #positive = list(positive)[:min_size]            # т.к. у нас довольно мало данных это сильно уменьшает
    #negative = list(negative)[:min_size]            # размер словаря и => точность классификатора
    
    print(f'No intersection: {set(positive).isdisjoint(set(negative))}')
    print(f'Positive: {len(positive)}')
    print(f'Negative: {len(negative)}')
                   
    return create_dict(positive, negative)

In [134]:
tone_dict = get_tone_words(train, 90)

Computing frequency dict...



No intersection: True
Positive: 276
Negative: 206


In [131]:
from sklearn.metrics import accuracy_score

In [132]:
def classify_review(review):
    review_class = Counter()
    words = preprocess(review)
    for word in words:
        if word in tone_dict:
            review_class[tone_dict[word]] += 1 
    return review_class.most_common()[0][0] if len(review_class) > 0 else 'pos'


def make_predictions(test):
    predictions = []
    for idx, x in test.iterrows():
        pred = classify_review(x['text'])
        predictions.append(pred)
    return predictions

In [135]:
prediction = make_predictions(test[['film_id', 'text']])
print(accuracy_score(prediction, test[['class']]))

0.67


Теперь со сбалансированной выборкой:

In [114]:
from sklearn.utils import shuffle


def balance_classes(data):
    balanced = []
    classes = data['class'].unique().tolist()
    size = data['class'].value_counts().min()
    for cl in classes:
        balanced.append(data[data['class']==cl].sample(size))
    return shuffle(pd.concat(balanced, ignore_index=True))
        

In [115]:
train_b = balance_classes(train)
test_b = balance_classes(test)

In [116]:
train_b['class'].value_counts()

neg    3999
pos    3999
Name: class, dtype: int64

In [117]:
test_b['class'].value_counts()

neg    999
pos    999
Name: class, dtype: int64

In [128]:
tone_dict = get_tone_words(train_b, 90)

Computing frequency dict...



No intersection: True
Positive: 276
Negative: 207


In [129]:
prediction = make_predictions(test_b[['film_id', 'text']])
print(accuracy_score(prediction, test_b[['class']]))

0.6706706706706707


Как видим, качество, конечно, улучшилось, но не очень сильно. Можно добавить к словарю больше разных коллокаций